$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Assignment 4: Classification with QDA, LDA, and Logistic Regression

Zach Goodenow <br>
3/27/18 <br>
CS 445 <br>

## Overview

In this assignment, I made a new version of my ```NeuralNetwork``` class which I called ```NeuralNetworkClassifier```.  I then applied QDA, LDA and my ```NeuralNetworkClassifier``` to a classification problem then discussed the results.  The ```tanh``` function was used as the activation function for ```NeuralNetworkClassifier```.

### NeuralNetworkClassifier

To start, I copied my ```neuralnetworksA2.py``` into a new file named ```neuralnetworksA4.py```.  I then defined a new class named ```NeuralNetworkClassifier``` that extends ```NeuralNetwork```.  The following code cell shows the methods that were created and override the parent class.  Comments were included in the assignment to instruct what was needed to complete it.  The class is also in my ```neuralnetworksA4.py``` but is pasted in the following cell for documentation purposes.

In [1]:
from neuralnetworksA4 import NeuralNetwork
import numpy as np

class NeuralNetworkClassifier(NeuralNetwork):

    def _multinomialize(self, Y):   # also known as softmax
        # fix to avoid overflow
        mx = max(0, np.max(Y))
        expY = np.exp(Y - mx)
        denom = np.sum(expY, axis=1).reshape((-1, 1)) + sys.float_info.epsilon
        return expY / denom

    def _objectiveF(self, w, X, Tindicators):
        self._unpack(w)
        # Do forward pass through all layers
        Zprev = X
        for i in range(len(self.nhs)):
            V = self.Vs[i]
            Zprev = self.activation(Zprev @ V[1:, :] + V[0:1, :])
        Y = Zprev @ self.W[1:, :] + self.W[0:1, :]

        # Convert Y to multinomial distribution. Call result G.
        G = self._multinomialize(Y)
        #  and return negative of mean log likelihood.
        # Write the call to np.log as  np.log(G + sys.float_info.epsilon)
        return -np.mean(Tindicators * np.log(G + sys.float_info.epsilon))


    def _gradientF(self, w, X, Tindicators):
        self._unpack(w)
        # Do forward pass through all layers
        Zprev = X
        Z = [Zprev]
        for i in range(len(self.nhs)):
            V = self.Vs[i]
            Zprev = self.activation(Zprev @ V[1:, :] + V[0:1, :])
            Z.append(Zprev)
        Y = Zprev @ self.W[1:, :] + self.W[0:1, :]

        # Convert Y to multinomial distribution. Call result G.
        G = self._multinomialize(Y)

        # delta calculated as before, except use Tindicators instead of T to
        #   make error of  Tindicates - G

        # Do backward pass, starting with delta in output layer
        delta = -(Tindicators - G) / (X.shape[0] * Tindicators.shape[1]) # MIGHT HAVE TO NEGATE THIS RESULT PER INSTRUCTIONS
        dW = np.vstack((np.ones((1, delta.shape[0])) @ delta,
                        Z[-1].T @ delta))
        dVs = []
        delta = self.activationDerivative(Z[-1]) * (delta @ self.W[1:, :].T)
        for Zi in range(len(self.nhs), 0, -1):
            Vi = Zi - 1  # because X is first element of Z
            dV = np.vstack((np.ones((1, delta.shape[0])) @ delta,
                            Z[Zi - 1].T @ delta))
            dVs.insert(0, dV)
            delta = (delta @ self.Vs[Vi][1:, :].T) * self.activationDerivative(Z[Zi - 1])
        return self._pack(dVs, dW)


    def train(self, X, T, nIterations=100, verbose=False,
              weightPrecision=0, errorPrecision=0, saveWeightsHistory=False):
        if self.Xmeans is None:
            self.Xmeans = X.mean(axis=0)
            self.Xstds = X.std(axis=0)
            self.Xconstant = self.Xstds == 0
            self.XstdsFixed = copy(self.Xstds)
            self.XstdsFixed[self.Xconstant] = 1
        X = self._standardizeX(X)

        if T.ndim == 1:
            T = T.reshape((-1, 1))

        # Remove T standardization calculations
        # Assign to Tindicators using ml.makeIndicatorVars
        Tindicators = ml.makeIndicatorVars(T)

        # Add   self.classes = np.unique(T)
        self.classes = np.unique(T)

        startTime = time.time()

        # Pass Tindicators into ml.scg instead of T
        scgresult = ml.scg(self._pack(self.Vs, self.W),
                           self._objectiveF, self._gradientF,
                           X, Tindicators,
                           xPrecision=weightPrecision,
                           fPrecision=errorPrecision,
                           nIterations=nIterations,
                           verbose=verbose,
                           ftracep=True,
                           xtracep=saveWeightsHistory)

        self._unpack(scgresult['x'])
        self.reason = scgresult['reason']
        self.errorTrace = np.sqrt(scgresult['ftrace'])  # * self.Tstds # to _unstandardize the MSEs
        self.numberOfIterations = len(self.errorTrace)
        self.trained = True
        self.weightsHistory = scgresult['xtrace'] if saveWeightsHistory else None
        self.trainingTime = time.time() - startTime
        return self

    def use(self, X, allOutputs=False):
        Zprev = self._standardizeX(X)
        Z = [Zprev]
        for i in range(len(self.nhs)):
            V = self.Vs[i]
            Zprev = self.activation(Zprev @ V[1:, :] + V[0:1, :])  # np.tanh(Zprev @ V[1:, :] + V[0:1, :])
            Z.append(Zprev)
        Y = Zprev @ self.W[1:, :] + self.W[0:1, :]

        # multinomialize Y, assign to G
        G = self._multinomialize(Y)
        # Calculate predicted classes by
        #   picking argmax for each row of G
        #   and use the results to index into self.classes.
        classes = np.array(self.classes[np.argmax(G, axis=1)]).reshape((-1,1))
        return (classes, G, Z[1:]) if allOutputs else classes

### Test It

In [2]:
import neuralnetworksA4 as nn
import imp
imp.reload(nn)
import mlutilities as ml
imp.reload(ml)
import numpy as np

In [116]:
X = np.arange(10).reshape((-1, 1))
T = np.array([1]*5 + [2]*5).reshape((-1, 1))

In [4]:
%pdb off

Automatic pdb calling has been turned OFF


In [5]:
netc = nn.NeuralNetworkClassifier(X.shape[1], [5, 5], len(np.unique(T)))
netc.train(X, T, 20)
print(netc)
print('T, Predicted')
print(np.hstack((T, netc.use(X))))

NeuralNetwork(1, [5, 5], 2)
   Network was trained for 21 iterations that took 0.0632 seconds. Final error is 0.006565536278730921.
T, Predicted
[[1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [2 2]
 [2 2]
 [2 2]
 [2 2]
 [2 2]]


## Addition to ```partition``` function

I added the keyword parameter ```classification``` with a default value of ```False``` to my ```partition``` function from assignment 3.  When its value is set to ```True``` my ```partition``` function performs a stratified partitioning as illustrated in lecture notes [12 Introduction to Classification](http://nbviewer.ipython.org/url/www.cs.colostate.edu/~anderson/cs445/notebooks/12%20Introduction%20to%20Classification.ipynb)     

I used my ```partition``` function from A3 and renamed it ```partitionHelper```.  I then created a new function named ```partition``` which has the exact same functionality when ```classification``` is set to ```False```.  

When ```classification``` is set to ```True```, it simply gathers ```Xtrain```, ```Ttrain```, ```Xtest```, and ```Ttest``` for every unique class in ```T``` by applying the helper function to each data set of classes separately.  At the end, each list of ```Xtrain```, ```Ttrain```, ```Xtest```, and ```Ttest``` are vertically stacked and returned.

I put both of these functions into ```mlutilities.py``` but put them in the following cell for documentation purposes.

In [6]:
# If shuffle=True is used as an argument, then the samples are randomly rearranged before the partitions are formed.
def partitionHelper(X, T, trainf, shuffle=False):
    nRows = X.shape[0]
    rows = np.arange(nRows)  # used as indices
    if shuffle:
        np.random.shuffle(rows)
    nTrain = round(nRows * trainf) # int(nRows * trainf) # IF YOU USE ROUND INSTEAD OF INT THIS WILL ROUND UP NOT DOWN
    trainRows = rows[:nTrain]
    testRows = rows[nTrain:]
    Xtrain, Ttrain = X[trainRows, :], T[trainRows, :]
    Xtest, Ttest = X[testRows, :], T[testRows, :]
    return Xtrain, Ttrain, Xtest, Ttest

def partition(X, T, trainf, classification=False, shuffle=False):
    if not classification:
        return partitionHelper(X, T, trainf, shuffle)
    compXtrain = []
    compTtrain = []
    compXtest = []
    compTtest = []
    for u in np.unique(T):
        Xr, Tr, Xe, Te = partitionHelper(X[np.where(T == u)[0]], T[np.where(T == u)[0]], trainf, shuffle)
        compXtrain.append(Xr)
        compTtrain.append(Tr)
        compXtest.append(Xe)
        compTtest.append(Te)
    return np.vstack(compXtrain), np.vstack(compTtrain), np.vstack(compXtest), np.vstack(compTtest)

In [140]:
import mlutilities as ml
imp.reload(ml);

In [117]:
Xtrain, Ttrain, Xtest, Ttest = ml.partition(X, T, 0.6, classification=True)

In [68]:
Xtrain

array([[0],
       [1],
       [2],
       [5],
       [6],
       [7]])

In [69]:
Ttrain

array([[1],
       [1],
       [1],
       [2],
       [2],
       [2]])

In [70]:
Xtest

array([[3],
       [4],
       [8],
       [9]])

In [71]:
Ttest

array([[1],
       [1],
       [2],
       [2]])

## Printing a confusion matrix

These two functions were provided in the assignment and I added them to my ```mlutilities.py``` file.

In [12]:
def confusionMatrix(actual, predicted, classes):
    nc = len(classes)
    confmat = np.zeros((nc, nc)) 
    for ri in range(nc):
        trues = (actual==classes[ri]).squeeze()
        predictedThisClass = predicted[trues]
        keep = trues
        predictedThisClassAboveThreshold = predictedThisClass
        # print 'confusionMatrix: sum(trues) is ', np.sum(trues),'for classes[ri]',classes[ri]
        for ci in range(nc):
            confmat[ri,ci] = np.sum(predictedThisClassAboveThreshold == classes[ci]) / float(np.sum(keep))
    printConfusionMatrix(confmat,classes)
    return confmat

def printConfusionMatrix(confmat,classes):
    print('   ',end='')
    for i in classes:
        print('%5d' % (i), end='')
    print('\n    ',end='')
    print('{:s}'.format('------'*len(classes)))
    for i,t in enumerate(classes):
        print('{:2d} |'.format(t), end='')
        for i1,t1 in enumerate(classes):
            if confmat[i,i1] == 0:
                print('  0  ',end='')
            else:
                print('{:5.1f}'.format(100*confmat[i,i1]), end='')
        print()

### Example with toy data

Use the above data to compare QDA, LDA, and linear and nonlinear logistic regression.

In [118]:
import qdalda
qda = qdalda.QDA()
qda.train(Xtrain, Ttrain)
Ytrain = qda.use(Xtrain)
Ytest = qda.use(Xtest)

In [119]:
print(np.hstack((Ttrain, Ytrain)))

[[1 1]
 [1 1]
 [1 1]
 [2 2]
 [2 2]
 [2 2]]


In [120]:
np.sum(Ttrain == Ytrain) / len(Ttrain) * 100

100.0

In [121]:
print(np.hstack((Ttest, Ytest)))

[[1 1]
 [1 2]
 [2 2]
 [2 2]]


In [122]:
np.sum(Ttest == Ytest) / len(Ttest) * 100

75.0

In [123]:
lda = qdalda.LDA()
lda.train(Xtrain, Ttrain)
Ytrain = lda.use(Xtrain)
Ytest = lda.use(Xtest)

In [124]:
print(np.hstack((Ttrain, Ytrain)))

[[1 1]
 [1 1]
 [1 1]
 [2 2]
 [2 2]
 [2 2]]


In [79]:
print(np.hstack((Ttest, Ytest)))

[[1 1]
 [1 2]
 [2 2]
 [2 2]]


In [125]:
np.sum(Ttrain == Ytrain) / len(Ttrain) * 100

100.0

In [126]:
np.sum(Ttest == Ytest) / len(Ttest) * 100

75.0

In [127]:
ml.confusionMatrix(Ttrain, Ytrain, [1, 2]);

Catch denom with: 3.0
Catch denom with: 3.0
Catch denom with: 3.0
Catch denom with: 3.0
       1    2
    ------------
 1 |100.0  0  
 2 |  0  100.0


In [83]:
ml.confusionMatrix(Ttest, Ytest, [1, 2]);

       1    2
    ------------
 1 | 50.0 50.0
 2 |  0  100.0


In [84]:
netc = nn.NeuralNetworkClassifier(X.shape[1], [5, 5], len(np.unique(T)))
netc.train(Xtrain, Ttrain, 100)
print(netc)
print('T, Predicted')
Ytrain = netc.use(Xtrain)
Ytest = netc.use(Xtest)

NeuralNetwork(1, [5, 5], 2)
   Network was trained for 101 iterations that took 0.0846 seconds. Final error is 6.083373583314762e-09.
T, Predicted


In [85]:
print(np.hstack((Ttrain, Ytrain)))

[[1 1]
 [1 1]
 [1 1]
 [2 2]
 [2 2]
 [2 2]]


In [86]:
print(np.hstack((Ttest, Ytest)))

[[1 1]
 [1 2]
 [2 2]
 [2 2]]


In [87]:
np.sum(Ttrain == Ytrain) / len(Ttrain) * 100

100.0

In [88]:
np.sum(Ttest == Ytest) / len(Ttest) * 100

75.0

In [89]:
ml.confusionMatrix(Ttrain, Ytrain, [1, 2]);

       1    2
    ------------
 1 |100.0  0  
 2 |  0  100.0


In [90]:
ml.confusionMatrix(Ttest, Ytest, [1, 2]);

       1    2
    ------------
 1 | 50.0 50.0
 2 |  0  100.0


Remember that linear logistic regression can be applied by specifying 0 hidden units.

In [91]:
netc = nn.NeuralNetworkClassifier(X.shape[1], 0, len(np.unique(T)))
netc.train(Xtrain, Ttrain, 100)
print(netc)
print('T, Predicted')
Ytrain = netc.use(Xtrain)
Ytest = netc.use(Xtest)

NeuralNetwork(1, [], 2)
   Network was trained for 101 iterations that took 0.0453 seconds. Final error is 1.218684296673873e-05.
T, Predicted


In [92]:
ml.confusionMatrix(Ttrain, Ytrain, [1, 2]);

       1    2
    ------------
 1 |100.0  0  
 2 |  0  100.0


In [93]:
ml.confusionMatrix(Ttest, Ytest, [1, 2]);

       1    2
    ------------
 1 | 50.0 50.0
 2 |  0  100.0


## Apply to data from orthopedic patients

I downloaded ```column_3C_weka.csv``` from [this Kaggle site](https://www.kaggle.com/uciml/biomechanical-features-of-orthopedic-patients).  The ```class``` column was used to create my target class labels. I applied QDA, LDA, linear logistic regression, and nonlinear logistic regression to this data.  I then experimented with different hidden layer structures and numbers of iterations.  Results and discussions are presented below including confusion matrices for training and for testing samples for each classifier.

In [49]:
import pandas as pd

In [209]:
odata = pd.read_csv('column_3C_weka.csv')
print(odata.shape)
odata[:5]

(310, 7)


pelvic_incidence  pelvic_tilt  lumbar_lordosis_angle  sacral_slope  \
0         63.027818    22.552586              39.609117     40.475232   
1         39.056951    10.060991              25.015378     28.995960   
2         68.832021    22.218482              50.092194     46.613539   
3         69.297008    24.652878              44.311238     44.644130   
4         49.712859     9.652075              28.317406     40.060784   

   pelvic_radius  degree_spondylolisthesis   class  
0      98.672917                 -0.254400  Hernia  
1     114.405425                  4.564259  Hernia  
2     105.985135                 -3.530317  Hernia  
3     101.868495                 11.211523  Hernia  
4     108.168725                  7.918501  Hernia

In [210]:
# Split data into target and input
odata = np.array(odata)
odata[:, 6].shape
T = odata[:, 6:] # Last column is class which is target
X = odata[:, :6].astype(np.float) # Rest of columns are X data
T.shape, X.shape

((310, 1), (310, 6))

In [211]:
np.unique(T)

array(['Hernia', 'Normal', 'Spondylolisthesis'], dtype=object)

</div> <div class="alert alert-block alert-warning">
Since the confusion matrix requires a number for classes and these classes are strings, I will represent "Hernia" as 1, "Normal" as 2, and "Spondylolisthesis" as 3 by replacing their values in the target matrix.

In [212]:
T[T == 'Hernia'] = 1
T[T == 'Normal'] = 2
T[T == 'Spondylolisthesis'] = 3
T.flatten()

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2,

Partition data into 80% for training and 20% for testing, with shuffle=True.

In [213]:
# Partition into 0.8 train shuffle=true
Xtrain, Ttrain, Xtest, Ttest = ml.partition(X, T, 0.8, classification=True, shuffle=True)

#### QDA

After training QDA on a few different partitions of the data, I found that it would sometimes result in very inaccurate train and test predictions.  Upon further investigation I found that when the model was very inaccurate it was because the model would only predict one class.  Hence, all samples of that one class would be correct and all others would be incorrect.  To avoid this error I partitioned the data and ran QDA until it created a model with predictions of all classes.  This could be called *Data dredging* and should be avoided in most cases.  I did this here so I could evaluate QDA against the other models but I know this is very dangerous and invalidates the results.

In [180]:
qda = qdalda.QDA()
qda.train(Xtrain, Ttrain)
Ytrain = qda.use(Xtrain)
Ytest = qda.use(Xtest)

In [181]:
print('QDA train accuracy', np.sum(Ttrain == Ytrain) / len(Ttrain) * 100)
ml.confusionMatrix(Ttrain, Ytrain, [1, 2, 3]);
print('QDA test accuracy',np.sum(Ttest == Ytest) / len(Ttest) * 100)
ml.confusionMatrix(Ttest, Ytest, [1, 2, 3]);

QDA train accuracy 86.2903225806
       1    2    3
    ------------------
 1 | 79.2 20.8  0  
 2 | 20.0 76.2  3.8
 3 |  0    4.2 95.8
QDA test accuracy 83.8709677419
       1    2    3
    ------------------
 1 | 75.0  8.3 16.7
 2 | 25.0 75.0  0  
 3 |  3.3  3.3 93.3


**DISCUSSION** <br>

**Train**: The model was correct about 86% of the time for training data.
<br> 
Confusion matrix shows:
- Hernia: accurately predicted for 79.2% of the data.  20.8% of the hernia data was misclassified as "Normal"
- Normal: accurately predicted for 76.2% of the data.  20% of the Normal data was misclassified as "Hernia" and 3.8% as "Spondylolisthesis"
- Spondylolisthesis: accurately predicted for 95.8% of the data.  4.2% of the Spondylolisthesis data was misclassified as "Normal"
<br><br>

**Test**: The model was correct about 86% of the time for testing data.
<br> 
Confusion matrix shows:
- Hernia: accurately predicted for 75% of the data.  8.3% of the hernia data was misclassified as "Normal" and 16.7% as "Spondylolisthesis"
- Normal: accurately predicted for 75% of the data.  25% of the Normal data was misclassified as "Hernia"
- Spondylolisthesis: accurately predicted for 93.3% of the data.  3.3% of the Spondylolisthesis data was misclassified as "Normal" an 3.3% as "Hernia"

#### LDA

In [183]:
lda = qdalda.LDA()
lda.train(Xtrain, Ttrain)
Ytrain = lda.use(Xtrain)
Ytest = lda.use(Xtest)

In [184]:
print('LDA train accuracy', np.sum(Ttrain == Ytrain) / len(Ttrain) * 100)
ml.confusionMatrix(Ttrain, Ytrain, [1, 2, 3]);
print('LDA test accuracy',np.sum(Ttest == Ytest) / len(Ttest) * 100)
ml.confusionMatrix(Ttest, Ytest, [1, 2, 3]);

LDA train accuracy 79.8387096774
       1    2    3
    ------------------
 1 | 81.2 18.8  0  
 2 | 21.2 72.5  6.2
 3 |  5.0 10.8 84.2
LDA test accuracy 83.8709677419
       1    2    3
    ------------------
 1 |100.0  0    0  
 2 | 35.0 65.0  0  
 3 |  3.3  6.7 90.0


**DISCUSSION** <br>

**Train**: The model was correct about 80% of the time for training data.
<br> 
Confusion matrix shows:
- Hernia: accurately predicted for 81.2% of the data.  18.8% of the hernia data was misclassified as "Normal"
- Normal: accurately predicted for 72.5% of the data.  21.2% of the Normal data was misclassified as "Hernia" and 6.2% as "Spondylolisthesis"
- Spondylolisthesis: accurately predicted for 84.2% of the data.  10.8% of the Spondylolisthesis data was misclassified as "Normal" and 5% as "Hernia"
<br><br>

**Test**: The model was correct about 84% of the time for testing data.
<br> 
Confusion matrix shows:
- Hernia: accurately predicted for 100% of the data.  WOW!
- Normal: accurately predicted for 65% of the data.  35% of the Normal data was misclassified as "Hernia"
- Spondylolisthesis: accurately predicted for 90% of the data.  6.7% of the Spondylolisthesis data was misclassified as "Normal" an 3.3% as "Hernia"

#### Linear Logistic Regression

Linear logistic regression can be applied by specifying 0 hidden units.  I will train the network for 10, 50, 100, and 200 iterations and evaluate each models accuracy using the confusion matrix.

In [220]:
for n in [10, 50, 100, 200]:
    netc = nn.NeuralNetworkClassifier(X.shape[1], 0, len(np.unique(T)))
    netc.train(Xtrain, Ttrain, n)
    Ytrain = netc.use(Xtrain)
    Ytest = netc.use(Xtest)
    print('--------------------------------------------------', n, '--------------------------------------------------')
    print(netc)
    print('Linear Logistic Regression train accuracy', np.sum(Ttrain == Ytrain) / len(Ttrain) * 100)
    ml.confusionMatrix(Ttrain, Ytrain, [1, 2, 3]);
    print('Linear Logistic Regression test accuracy',np.sum(Ttest == Ytest) / len(Ttest) * 100)
    ml.confusionMatrix(Ttest, Ytest, [1, 2, 3]);

-------------------------------------------------- 10 --------------------------------------------------
NeuralNetwork(6, [], 3)
   Network was trained for 11 iterations that took 0.0080 seconds. Final error is 0.3477052653374724.
Linear Logistic Regression train accuracy 87.0967741935
       1    2    3
    ------------------
 1 | 75.0 22.9  2.1
 2 | 15.0 81.2  3.8
 3 |  0.8  3.3 95.8
Linear Logistic Regression test accuracy 79.0322580645
       1    2    3
    ------------------
 1 | 58.3 41.7  0  
 2 | 15.0 70.0 15.0
 3 |  3.3  3.3 93.3
-------------------------------------------------- 50 --------------------------------------------------
NeuralNetwork(6, [], 3)
   Network was trained for 51 iterations that took 0.0255 seconds. Final error is 0.34397874154886154.
Linear Logistic Regression train accuracy 85.8870967742
       1    2    3
    ------------------
 1 | 66.7 31.2  2.1
 2 | 13.8 82.5  3.8
 3 |  0.8  3.3 95.8
Linear Logistic Regression test accuracy 87.0967741935
       1 

Each confusion matrix can be interpreted as was done for that of QDA and LDA.  I will leave that interpretation out to save space and talk about bigger picture interpretation.
<br><br>
The most accurate model looks like it was the one that was trained for 50 iterations.  The highest testing accuracy was 87.09677% in both 50 iterations and 100 iterations.  This model was very good a predicting Spondylolisthesis; over 90% accurate for all iterations and train/test.  This model seams to have a tough time predicting Hernia. It tends to misclassify Hernia data as "Normal" for all iterations.  Interestingly, Normal samples seam to be accurately classified 80-95% of the time.  This tells me that this linear logistic regression models will have a bias towards predicting "Normal" for both "Hernia" samples and "Normal" samples.  

#### Nonlinear Logistic Regression

For these models, I would like to evaluate different hidden layer structures.  Since the experiment for linear logistic regression showed that 100 training iterations produced a relatively accurate result, I will use 100 training iterations.  Ideally, multiple iterations should be evaluated for each hidden layer structure. 

In [222]:
hiddens = [[nu] * nl for nu in [1, 5, 10, 20, 50] for nl in [1, 2, 5, 10]]
for hid in hiddens:
    netc = nn.NeuralNetworkClassifier(X.shape[1], hid, len(np.unique(T)))
    netc.train(Xtrain, Ttrain, 100)
    Ytrain = netc.use(Xtrain)
    Ytest = netc.use(Xtest)
    print('--------------------------------------------------', hid, '--------------------------------------------------')
    print('Nonlinear Logistic Regression train accuracy', np.sum(Ttrain == Ytrain) / len(Ttrain) * 100)
    ml.confusionMatrix(Ttrain, Ytrain, [1, 2, 3]);
    print('Nonlinear Logistic Regression test accuracy',np.sum(Ttest == Ytest) / len(Ttest) * 100)
    ml.confusionMatrix(Ttest, Ytest, [1, 2, 3]);

-------------------------------------------------- [1] --------------------------------------------------
Nonlinear Logistic Regression train accuracy 79.8387096774
       1    2    3
    ------------------
 1 | 43.8 56.2  0  
 2 | 18.8 76.2  5.0
 3 |  0    3.3 96.7
Nonlinear Logistic Regression test accuracy 79.0322580645
       1    2    3
    ------------------
 1 | 33.3 66.7  0  
 2 |  5.0 85.0 10.0
 3 |  0    6.7 93.3
-------------------------------------------------- [1, 1] --------------------------------------------------
Nonlinear Logistic Regression train accuracy 80.6451612903
       1    2    3
    ------------------
 1 | 45.8 54.2  0  
 2 | 18.8 76.2  5.0
 3 |  0    2.5 97.5
Nonlinear Logistic Regression test accuracy 77.4193548387
       1    2    3
    ------------------
 1 | 33.3 66.7  0  
 2 | 10.0 80.0 10.0
 3 |  0    6.7 93.3
-------------------------------------------------- [1, 1, 1, 1, 1] --------------------------------------------------
Nonlinear Logistic Regres

-------------------------------------------------- [50, 50, 50, 50, 50, 50, 50, 50, 50, 50] --------------------------------------------------
Nonlinear Logistic Regression train accuracy 89.1129032258
       1    2    3
    ------------------
 1 | 85.4 14.6  0  
 2 | 20.0 77.5  2.5
 3 |  0.8  0.8 98.3
Nonlinear Logistic Regression test accuracy 85.4838709677
       1    2    3
    ------------------
 1 | 83.3 16.7  0  
 2 | 25.0 65.0 10.0
 3 |  0    0  100.0


Each confusion matrix can be interpreted as was done for that of QDA and LDA.  I will leave that interpretation out to save space and talk about bigger picture interpretation.
<br><br>
The most accurate models were those with hidden layer structures of [10], [10, 10, 10, 10, 10, 10, 10, 10, 10, 10],  or, [50, 50, 50, 50, 50, 50, 50, 50, 50, 50] with about 85% test accuracy.  These models still dont preform as well as they could so maybe they need to be trained for more iterations.  They seam to classify Normal data more accurately than that of linear logistic regression and Hernia data less accurately.  The models tend to lean towards classifiying sample data as Hernia rather than Normal which is interesting because it was the other way around for linear logistic regression.
<br><br>
When it comes to Spondylolisthesis samples, these models were extremely accurate in both training and testing for all hidden layer structures.  Most of the models produced high 90s or even 100% accuracy for Spondylolisthesis predictions.  Additionally, these models rarely misclassified "Normal" or "Hernia" samples as Spondylolisthesis.  This tells me that if I was trying to predict Spondylolisthesis for a given sample, I would use a Nonlinear Logistic Regression model.  However, this model seams to have a tough time spotting the difference between Normal and Hernia data.  
<br>
Another observation I make from this experiment is that the training accuracy is typically more accurate than testing accuracy.  In some cases it is over 20% more accurate.  This behavior is something that wasn't present in the other model types.  This could mean that the nonlinear logistic regression models are overfitting the train data, resulting in inaccurate test results.  It might be beneficial to train these models for less iterations to avoid overfitting.  

## Grading and Check-in

Your notebook will be run and graded automatically. Test this grading process by first downloading [A4grader.tar](https://www.cs.colostate.edu/~anderson/cs445/notebooks/A4grader.tar) and extract A4grader.py from it.   

In [198]:
%run -i A4grader.py


======================= Code Execution =======================

Extracting python code from notebook named 'Goodenow-A4.ipynb' and storing in notebookcode.py
Removing all statements that are not function or class defs or import statements.

Testing  import neuralnetworksA4 as nn

--- 5/5 points. The statement  import neuralnetworksA4 as nn  works.

Testing  import mlutilities as ml

--- 5/5 points. The statement  import mlutilities as ml  works.

Testing X = np.vstack((np.arange(20), [7, 4, 5, 5, 8, 4, 6, 7, 4, 9, 4, 2, 6, 6, 3, 3, 7, 2, 6, 4])).T
        T = np.array([1]*8 + [2]*8 + [3]*4).reshape((-1, 1))
        Xtrain, Ttrain, Xtest, Ttest = ml.partition(X, T, 0.8, classification=True, shuffle=False)

--- 10/10 points. ml.partition works correctly

Testing nnet = nn.NeuralNetworkClassifier(2, [5, 5], 3)

--- 10/10 points. nn.NeuralNetworkClassifier did not throw exception.

Testing nnet.train(Xtrain, Ttrain, 200)

--- 20/20 points. nnet.getErrors()[-1] is less than 1.e-5.

Testing

# Extra Credit

Earn 1 extra credit point by doing a few experiments with different neural network classifiers using the ReLU activation function on the orthopedic data. Discuss any differences you see from your earlier results that used tanh.